## Install Library

Install hnswlib terlebih dahulu.

In [3]:
!pip install hnswlib

## Percobaan

Percobaan berikut akan membandingkan exact NN dengan HNSW pada 1000 data 2D.

In [4]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)  

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

=== Exact NN ===
Indices: [[829 246 472 912 591]]
Distances: [[0.00993625 0.01363944 0.01646171 0.03779694 0.03859071]]
Waktu: 0.15031790733337402 detik

=== HNSW ===
Indices: [[829 246 472 912 591]]
Distances: [[9.8728990e-05 1.8603442e-04 2.7098786e-04 1.4286089e-03 1.4892431e-03]]
Waktu: 0.00034165382385253906 detik


## Tugas

Lakukan percobaan pada metric distance yang berbeda, 1000 vs 1jt data, 2D vs 5D data. Catat hasilnya pada tabel yang anda buat sendiri seperti pada praktikum 1.

In [5]:
import pandas as pd

# Fungsi untuk menjalankan eksperimen
def run_experiment_hnsw(data, query, k=10):
    dim = data.shape[1]

    # Exact NN
    nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
    nn.fit(data)
    start = time.time()
    distances_exact, indices_exact = nn.kneighbors(query)
    time_exact = time.time() - start

    # HNSW
    p = hnswlib.Index(space='l2', dim=dim)
    p.init_index(max_elements=len(data), ef_construction=100, M=16)
    p.add_items(data)
    p.set_ef(50)

    start = time.time()
    labels, distances_hnsw = p.knn_query(query, k=k)
    time_hnsw = time.time() - start

    # Hitung recall
    exact_set = set(indices_exact[0])
    hnsw_set = set(labels[0])
    recall = len(exact_set & hnsw_set) / k

    return {
        'time_exact': time_exact,
        'time_hnsw': time_hnsw,
        'recall': recall
    }

# Konfigurasi eksperimen
configs = [
    {'n': 1000, 'd': 2},
    {'n': 1000000, 'd': 2},
    {'n': 1000, 'd': 5},
    {'n': 1000000, 'd': 5}
]

results = []

for config in configs:
    n, d = config['n'], config['d']
    np.random.seed(42)
    data = np.random.random((n, d)).astype(np.float32)
    query = np.random.random((1, d)).astype(np.float32)

    res = run_experiment_hnsw(data, query)
    results.append({
        'Data Size': n,
        'Dimensions': d,
        'Exact Time (s)': f"{res['time_exact']:.6f}",
        'HNSW Time (s)': f"{res['time_hnsw']:.6f}",
        'Recall@10': f"{res['recall']:.3f}"
    })

# Buat tabel
df = pd.DataFrame(results)
print(df.to_string(index=False))

 Data Size  Dimensions Exact Time (s) HNSW Time (s) Recall@10
      1000           2       0.003947      0.000059     1.000
   1000000           2       0.007791      0.000044     1.000
      1000           5       0.005658      0.000045     1.000
   1000000           5       0.013048      0.000054     1.000
